In [1]:
import pandas as pd

from pathlib import Path
import utils.data_loader as data_loader
import utils.model_loader as model_loader
import utils.predictions as predictions

from surprise import SVD, NMF
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.algo_base import AlgoBase
from surprise.dump import dump, load


    
def get_user_recommendation(model: AlgoBase, user_id: int, k: int) -> pd.DataFrame:
    """Makes movie recommendations a user.
    
    Parameters
    ----------
        model : AlgoBase
            A trained surprise model
        user_id : int
            The user for whom the recommendation will be done.
        k : int
            The number of items to recommend.
        
    Returns
    -------
    pandas.Dataframe
        A dataframe with the k movies that will be recommended the user. The dataframe should have the following
        columns (movie_name : str, movie_genre : str, predicted_rating : float, true_rating : float)
        
    Notes
    -----
    - You should create other functions that are used in this one and not put all the code in the same function.
        For example to create the final dataframe, instead of implemented all the code
        in this function (get_user_recommendation), you can create a new one (create_recommendation_dataframe)
        that will be called in this function.
    - You can add other arguments to the function if you need to.
    """
    
    data_dir = Path(Path.cwd().parents[0], 'data', 'movielens', 'ml-latest-small')
    model_dir = Path(Path.cwd().parents[0], 'models')
    
    try:
        preds, model = load(Path(model_dir, str(model)[8:-2]))
    except:
        data = data_loader.get_data('ml-100k')
        trainset = data.build_full_trainset()
        testset = trainset.build_anti_testset()
        trained_model = model_loader.get_trained_model(model, trainset)
        preds = trained_model.test(testset)
        dump(Path(model_dir, str(model)[8:-2]), preds, trained_model)
    top_n = predictions.get_top_n(preds, n=k)
    movies = data_loader.get_item_data('movies.csv', data_dir)

    
    return predictions.get_item_details(top_n[user_id], movies, 'movieId', ['movieId', 'title', 'genres'])

recommendations = get_user_recommendation(SVD, '196', 10)
recommendations.reset_index(drop=True, inplace=True)
display(recommendations)

,movieId,title,genres,tru_r,est_r
0,12,Dracula: Dead and Loving It (1995),Comedy|Horror,3.52986,4.472080
1,64,Two if by Sea (1996),Comedy|Romance,3.52986,4.437692
2,178,Love & Human Remains (1993),Comedy|Drama,3.52986,4.455130
3,318,"Shawshank Redemption, The (1994)",Crime|Drama,3.52986,4.593318
4,408,8 Seconds (1994),Drama,3.52986,4.474681
5,479,Judgment Night (1993),Action|Crime|Thriller,3.52986,4.417899
6,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,3.52986,4.466937
